# 01 - Initial EDA for Steam Web & Store data 📊

This notebook performs a first exploratory data analysis (EDA) on the data that has been
ingested so far:

- `data/bronze/steam_web/app_list.parquet` - global Steam app list (appid, name, etc.).
- `data/bronze/steam_store/app_details.parquet` - Steam Store app details (incremental).

The goal is to get a feeling for data volume, schema, and a few basic distributions that we
can later use for feature engineering and ML experiments.


In [6]:
from pathlib import Path

import pandas as pd

# Assume this notebook lives in `notebooks/` under the project root.
PROJECT_ROOT = Path('..').resolve()
DATA_DIR = PROJECT_ROOT / 'data'
WEB_PARQUET = DATA_DIR / 'bronze' / 'steam_web' / 'app_list.parquet'
STORE_PARQUET = DATA_DIR / 'bronze' / 'steam_store' / 'app_details.parquet'

WEB_PARQUET, STORE_PARQUET

(WindowsPath('C:/dev/steam-ml/data/bronze/steam_web/app_list.parquet'),
 WindowsPath('C:/dev/steam-ml/data/bronze/steam_store/app_details.parquet'))

## 1. Steam Web app list


In [7]:
if not WEB_PARQUET.exists():
    raise FileNotFoundError(
        f'Expected app list parquet at {WEB_PARQUET}, but it does not exist. '
        'Please run `run_steam_web_ingest.py` first.'
    )

apps_df = pd.read_parquet(WEB_PARQUET)

print('App list shape (rows, cols):', apps_df.shape)
print('Columns (first 20):')
print(list(apps_df.columns)[:20])

apps_df.head()

App list shape (rows, cols): (208029, 4)
Columns (first 20):
['appid', 'name', 'last_modified', 'price_change_number']


,appid,name,last_modified,price_change_number
0,10,Counter-Strike,1745368572,32873429
1,20,Team Fortress Classic,1745368565,32873429
2,30,Day of Defeat,1745368580,32873429
3,40,Deathmatch Classic,1745368570,32873429
4,50,Half-Life: Opposing Force,1745368539,32873429


In [8]:
apps_df.describe(include='all').T.head(20)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
appid,208029.0,NaN,NaN,NaN,2107038.518533,1134608.507054,10.0,1142270.0,2051650.0,3059750.0,4285180.0
name,208029,206232,,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last_modified,208029.0,NaN,NaN,NaN,1697126686.661028,75874464.172584,1447350914.0,1663843014.0,1726781620.0,1756230731.0,1767621690.0
price_change_number,208029.0,NaN,NaN,NaN,24562266.777454,12292645.954625,0.0,22736671.0,31509480.0,32873429.0,33075455.0


In [9]:
# Convert last_modified (epoch seconds) to datetime if present
if 'last_modified' in apps_df.columns:
    apps_df['last_modified_dt'] = pd.to_datetime(apps_df['last_modified'], unit='s', errors='coerce')
    print('last_modified_dt range:')
    print(apps_df['last_modified_dt'].min(), '->', apps_df['last_modified_dt'].max())
else:
    print('Column `last_modified` not present in app list dataframe.')


last_modified_dt range:
2015-11-12 17:55:14 -> 2026-01-05 14:01:30


## 2. Steam Store app details (if available)


In [10]:
if not STORE_PARQUET.exists():
    print(f'No store details parquet found at {STORE_PARQUET}. Run `run_steam_store_ingest.py` first.')
    store_df = None
else:
    store_df = pd.read_parquet(STORE_PARQUET)
    print('Store details shape (rows, cols):', store_df.shape)
    print('Columns (first 20):')
    print(list(store_df.columns)[:20])

store_df.head() if store_df is not None else None

Store details shape (rows, cols): (595, 128)
Columns (first 20):
['type', 'name', 'steam_appid', 'required_age', 'is_free', 'detailed_description', 'about_the_game', 'short_description', 'supported_languages', 'header_image', 'capsule_image', 'capsule_imagev5', 'website', 'developers', 'publishers', 'packages', 'package_groups', 'categories', 'genres', 'screenshots']


,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,...,ratings.fpb.use_age_gate,ratings.fpb.required_age,ext_user_account_notice,ratings.cero.rating,ratings.cero.use_age_gate,ratings.cero.required_age,ratings.cero.descriptors,ratings.fpb.descriptors,ratings.bbfc.descriptors,ratings.crl.descriptors
0,game,Counter-Strike,10,0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://shared.akamai.steamstatic.com/store_it...,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,game,Team Fortress Classic,20,0,False,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://shared.akamai.steamstatic.com/store_it...,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,game,Day of Defeat,30,0,False,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://shared.akamai.steamstatic.com/store_it...,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,game,Deathmatch Classic,40,0,False,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://shared.akamai.steamstatic.com/store_it...,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,game,Half-Life: Opposing Force,50,0,False,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,"English, French, German, Korean",https://shared.akamai.steamstatic.com/store_it...,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [11]:
if store_df is not None:
    # Basic summary for a few likely interesting columns, if they exist
    summary_cols = [
        'appid',
        'name',
        'type',
        'required_age',
        'is_free',
    ]

    # Add any price_overview.* columns that may exist
    price_cols = [c for c in store_df.columns if c.startswith('price_overview')]
    summary_cols.extend(price_cols)

    present_cols = [c for c in summary_cols if c in store_df.columns]
    print('Summary columns present:', present_cols)

    store_df[present_cols].describe(include='all').T.head(20)
else:
    None

Summary columns present: ['appid', 'name', 'type', 'required_age', 'is_free', 'price_overview.currency', 'price_overview.initial', 'price_overview.final', 'price_overview.discount_percent', 'price_overview.initial_formatted', 'price_overview.final_formatted']


## 3. Join Web app list and Store details


In [12]:
if store_df is not None:
    merged = apps_df.merge(
        store_df,
        on='appid',
        how='left',
        suffixes=('_web', '_store'),
    )
    print('Merged shape (rows, cols):', merged.shape)
    print('Rows with store details:', merged['name_store'].notna().sum() if 'name_store' in merged.columns else 'n/a')
else:
    merged = None
    print('Store details not available; skipping merge.')

Merged shape (rows, cols): (208029, 132)
Rows with store details: 595


In [13]:
if merged is not None:
    # Simple sanity checks: free vs paid, by type
    if 'is_free' in merged.columns and 'type' in merged.columns:
        ctab = (
            merged
            .groupby(['type', 'is_free'])['appid']
            .nunique()
            .unstack(fill_value=0)
            .rename_axis(index='type', columns='is_free')
        )
        print('Unique appids by type and is_free:')
        display(ctab)
    else:
        print('Columns `type` and/or `is_free` not present in merged dataframe.')


Unique appids by type and is_free:


is_free,False,True
type,,
dlc,81,1
game,493,20


## 4. Next EDA ideas

Some directions you can extend from here:

- Plot distributions (owners, price, discount, required_age) once more columns are available.
- Look at correlations between basic numeric features.
- Parse and explode tag/genre columns from the store data string fields.
- Build feature tables ("silver" / "gold") for ML experiments (success prediction, recommendations, etc.).
